Nesta aula, provavelmente será necessário usar GPU para facilitar as coisas. Provavelmente o ambiente será migrado para o Colab. No entanto, começarei fazendo aqui para ver como é o desempenho da CPU e ao final, apenas importo o arquivo .ipynb para o Colab.

In [10]:
import torch
from torch import nn, optim # nn possui as funcoes de perda, optim os otimizadores
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np


# Criando um dicionario
args = {
    'batch_size': 500,
    'num_workers': 4, # numero de threads
    'lr': 1e-4,
    'weight_decay': 5e-4,
    'num_epochs': 30 
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

cpu


In [11]:
train_set = datasets.MNIST('./',
                           train = True,
                           transform = transforms.ToTensor(),
                           # transform = transforms.RandomCrop(12),
                           download = True)
test_set = datasets.MNIST('./',
                           train = False,
                           transform = transforms.ToTensor(),
                           download = False)

print('Amostras de treino: ' + str(len(train_set)) + '\nAmostras de teste: ' + str(len(test_set)))

Amostras de treino: 60000
Amostras de teste: 10000


In [12]:
# Quando formos trabalhar com datasets proprios, teremos que implementar classes especificas dessa mesma forma
print(type(train_set), type(test_set))
print(type(train_set[0]), type(test_set[0]))

# Sempre sera uma tupla(dado, rotulo)
print(train_set[0])

<class 'torchvision.datasets.mnist.MNIST'> <class 'torchvision.datasets.mnist.MNIST'>
<class 'tuple'> <class 'tuple'>
(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.

In [13]:
for i in range(3):
    dado, rotulo = train_set[i]
    # dado, rotulo = train_set[0]

    plt.figure()
    plt.imshow(dado[0])
    # plt.imshow(dado)
    plt.title('Rotulo' + str(rotulo))
    plt.show()

invalid command name "140523292149312process_stream_events"
    while executing
"140523292149312process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [14]:
train_loader = DataLoader(train_set, batch_size=args['batch_size'], shuffle=True, num_workers=args['num_workers'])

test_loader = DataLoader(test_set, batch_size=args['batch_size'], shuffle=True, num_workers=args['num_workers'])

In [15]:
# Iterador, mas nao suporta indexacao
for batch in train_loader:
    dado, rotulo = batch
    print(dado.size(), rotulo.size())

    plt.imshow(dado[0][0])
    plt.title('Rotulo: ' + str(rotulo[0]))
    plt.show()
    break

torch.Size([500, 1, 28, 28]) torch.Size([500])


In [16]:
class MLP(nn.Module):

    def __init__(self, input_size, hidden_size, out_size):
        super(MLP, self).__init__()

        self.features = nn.Sequential(
                            nn.Linear(input_size, hidden_size),
                            nn.ReLU(),
                            nn.Linear(hidden_size, hidden_size),
                            nn.ReLU()
                        )
        self.out = nn.Linear(hidden_size, out_size)
        self.softmax = nn.Softmax()

    def forward (self, X):

        X = X.view(X.size(0), -1)

        feature = self.features(X)
        output = self.softmax(self.out(feature))

        return output
    
input_size = 28 * 28
hidden_size = 128 # numero escolhido arbitrariamente
out_size = 10 # numero de classes

net = MLP(input_size, hidden_size, out_size).to(args['device']) # cast na GPU

In [17]:
criterion = nn.CrossEntropyLoss().to(args['device'])
optimizer = optim.Adam(net.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

In [18]:
for epoch in range(args['num_epochs']):

    epoch_loss = []

    for batch in train_loader:
        dado, rotulo = batch
        dado = dado.to(args['device'])
        rotulo = rotulo.to(args['device'])

        # Forward
        pred = net(dado)
        loss = criterion(pred, rotulo)
        epoch_loss.append(loss.cpu().data)

        # Backward
        loss.backward()
        optimizer.step()
    
    epoch_loss = np.asarray(epoch_loss)

    print("Epoca: %d, Loss: %.4f +\- %.4f" % (epoch, epoch_loss.mean(), epoch_loss.std()))

# batch_size: 20
# Aproximadamente 30 segundos entre a epoca 3 e 4.
# 2 minutos e 37 segundos para 4 epocas
# Epocas para loss abaixo de 1.6: mesmo apos 11 epocas (5 min 43 s), o desvio padrao foi diminuindo e a media aumentando

# batch_size: 100
# Epocas para loss abaixo de 1.6: epocas finalizadas em 29 (0 - 29), nao ficou abaixo de 1.6
# 3 min e 50 segundos para 30 epocas

# batch_size: 5
# Epocas para loss abaixo de 1.6: mesmo apos 2 epocas (5 min 54 s), a media foi aumentando
# 5 min e 54 segundos para 3 epocas (0 - 2)     

# Percebe-se pelas notas da professora, que usando a GPU, o tempo mais rapido foi de 5 segundos por epoca e o mais lento de 38 segundos por epoca. Diferentemente da CPU local, que o mais rapido foi de quase 7,6 segundos por epoca e o mais lento de 118 segundos por epoca

# Apenas com batch_size: 500, que finalmente foi possivel ter uma loss abaixo de 1.6, demorando 9 epocas. Ao rodar as 30 epocas em 2 min 40 s, temos uma media de 5.3 s por epoca

Epoca: 0, Loss: 2.2302 +\- 0.0734
Epoca: 1, Loss: 1.9293 +\- 0.0675
Epoca: 2, Loss: 1.8043 +\- 0.0309
Epoca: 3, Loss: 1.7336 +\- 0.0217
Epoca: 4, Loss: 1.6968 +\- 0.0285
Epoca: 5, Loss: 1.6880 +\- 0.0214
Epoca: 6, Loss: 1.6356 +\- 0.0194
Epoca: 7, Loss: 1.6185 +\- 0.0167
Epoca: 8, Loss: 1.6090 +\- 0.0174
Epoca: 9, Loss: 1.5968 +\- 0.0135
Epoca: 10, Loss: 1.5798 +\- 0.0154
Epoca: 11, Loss: 1.5800 +\- 0.0132
Epoca: 12, Loss: 1.5741 +\- 0.0150
Epoca: 13, Loss: 1.5694 +\- 0.0128
Epoca: 14, Loss: 1.5623 +\- 0.0128
Epoca: 15, Loss: 1.5593 +\- 0.0119
Epoca: 16, Loss: 1.5554 +\- 0.0136
Epoca: 17, Loss: 1.5538 +\- 0.0119
Epoca: 18, Loss: 1.5519 +\- 0.0136
Epoca: 19, Loss: 1.5474 +\- 0.0130
Epoca: 20, Loss: 1.5474 +\- 0.0133
Epoca: 21, Loss: 1.5459 +\- 0.0131
Epoca: 22, Loss: 1.5410 +\- 0.0120
Epoca: 23, Loss: 1.5391 +\- 0.0107
Epoca: 24, Loss: 1.5409 +\- 0.0121
Epoca: 25, Loss: 1.5395 +\- 0.0112
Epoca: 26, Loss: 1.5392 +\- 0.0115
Epoca: 27, Loss: 1.5386 +\- 0.0137
Epoca: 28, Loss: 1.5360 +\- 0.